# 数据检索

用户输入只是问题，答案需要从已知的数据库或者文档中获取。因此，我们需要使用检索器获取上下文，并通过“question”键下的用户输入结合上下文来回答问题。

### RAG
检索增强生成（Retrieval-augmented Generation，RAG），是当下最热门的大模型前沿技术之一。如果将“微调（finetune）”理解成大模型内化吸收知识的过程，那么RAG就相当于给大模型装上了“知识外挂”，基础大模型不用再训练即可随时调用特定领域知识。

In [2]:
# 导入本地词嵌入模型
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="/Users/libing/kk_LLMs/bge-large-zh-v1.5")

In [3]:
# 构建上下文向量数据库
texts = [
    "今天天气很好，适合出去游玩。", 
    "今天天气不好，不适合出去游玩。",
    "小明在华为工作，是一个非常优秀的青年。",
    '熊喜欢吃蜂蜜，尤其是新鲜的蜂蜜。',
    '狗喜欢吃屎，尤其是热乎乎的新鲜的狗屎。'
]
vector_store = FAISS.from_texts(texts, embeddings)
vector_store

In [4]:
# 使用向量数据库生成检索器
retriever = vector_store.as_retriever()

In [5]:
retriever.invoke("今天天气怎么样？")

[Document(metadata={}, page_content='今天天气很好，适合出去游玩。'),
 Document(metadata={}, page_content='今天天气不好，不适合出去游玩。'),
 Document(metadata={}, page_content='狗喜欢吃屎，尤其是热乎乎的新鲜的狗屎。'),
 Document(metadata={}, page_content='熊喜欢吃蜂蜜，尤其是新鲜的蜂蜜。')]

In [6]:
# 接入本地部署的大模型
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)


In [7]:
# 构建提示词模板
template = """
请根据以下文档回答问题:
{context}

问题:
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [9]:
# 构建链
from langchain_core.runnables import RunnablePassthrough

output_parser = StrOutputParser()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

chain.invoke("今天不错，是否适合出去游玩？")

'根据提供的文档内容，有两份关于天气情况的信息，其中一份表示“今天天气很好，适合出去游玩”，而另一份则说“今天天气不好，不适合出去游玩”。因此，需要依据实际的天气状况来判断。如果今天的天气确实很好，则适合出去游玩；反之，则不太适合。由于文档中存在矛盾的信息，建议您查看最新的天气预报以做出决定。'

In [10]:
chain.invoke("狗熊喜欢吃吃什么？")

'根据提供的文档内容，“熊喜欢吃蜂蜜，尤其是新鲜的蜂蜜。”因此，熊喜欢吃蜂蜜。\n\n关于“狗”的饮食偏好，在这里仅作为对比信息提供，并不是关于狗熊（假设您指的是熊）的问题答案。所以，针对您的问题，熊喜欢吃的是蜂蜜。'

In [13]:
# 在回答问题的时候加上称呼
from operator import itemgetter

new_template = """\
只根据以下上下文回答问题: 
{context}

问题: {question}
回答问题的时候请加上称呼: {name}    
"""
new_prompt = ChatPromptTemplate.from_template(new_template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "name": itemgetter("name")
    }
    | new_prompt
    | llm
    | output_parser
)

chain.invoke({"question": "小明在哪里工作", "name": "主人"})

'主人，小明在华为工作。'